In [1]:
import pandas as pd
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input
from keras.models import Model
from keras.layers.normalization import BatchNormalization
import time
from IPython import display
import math
import time
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def minMaxNormalise(data, cont_ls):
    for j in cont_ls:
        index = list(data.index.values)
        df_ = pd.DataFrame(index=index, columns=[j])
        for i in index:
            df_.at[i,j] = (data.at[i,j]-data[j].min())/(data[j].max()-data[j].min())
        data.at[:,j] = np.nan
        for k in index:
            data.at[k,j] = df_.at[k,j]
    
    return data

In [3]:
def meanNormalise(data, cont_ls):
    for j in cont_ls:
        index = list(data.index.values)
        df_ = pd.DataFrame(index=index, columns=[j])
        for i in index:
            df_.at[i,j] = (data.at[i,j]-data[j].mean())/(data[j].std())
        data.at[:,j] = np.nan
        for k in index:
            data.at[k,j] = df_.at[k,j]
    
    return data

In [4]:
def oneHotEnc(data, cat_ls):
    for i in cat_ls:
        col_df = data[i]
        encoded_df = pd.get_dummies(col_df, prefix = [i])
        data = data.drop([i], axis=1)
        data = pd.concat([data, encoded_df], axis = 1, sort=False)
    return data

In [5]:
def sample_z(m, n):
    seed = np.random.uniform(0., 1., size=[m, n])
    seed = np.sort(seed)
    return seed

In [65]:
data = pd.read_csv('heart-disease-uci.zip')
data_process = False
if data_process:    
    cont_ls = ['age', 'trestbps', 'chol', 'thalach','oldpeak']
    data = minMaxNormalise(data, cont_ls)
    cat_col = ['cp', 'slope', 'thal']
    data = oneHotEnc(data, cat_col)

target_df = data['target']
variables = data.drop(['target'], axis=1)
variables = variables.drop(['cp', 'slope', 'thal'], axis=1)

data_train, data_test, label_train, label_test = train_test_split(variables, target_df, test_size=0.20)
gan_data = pd.concat([data_train, label_train], axis=1, sort=False)
print(gan_data.shape)
print(label_test.shape)

logreg = LogisticRegression()
logreg.fit(data_train, label_train)
accuracy = round(logreg.score(data_test, label_test)*100, 2)
print(accuracy)
print(data.shape)
print(target_df.describe())
print(data_train.shape)

(242, 11)
(61,)
80.33
(303, 14)
count    303.000000
mean       0.544554
std        0.498835
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: target, dtype: float64
(242, 10)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
tf.keras.backend.clear_session
tf.reset_default_graph()
data = pd.read_csv('heart-disease-uci.zip')
cat_col = ['cp', 'slope', 'thal', 'target']
data_cat = data[cat_col]
data_cat = oneHotEnc(data_cat, cat_col)
dummy_labels = np.ones([data_cat.shape[0]])
print(data_cat.shape)
print(dummy_labels.shape)
data_train, data_test, _, __, = train_test_split(data_cat, dummy_labels, test_size = 0.1)

encoding_dim = 40
input_ = Input(shape=(data_cat.shape[1], ))
encoded = Dense(encoding_dim, activation='relu')(input_)
decoding_layer = Dense(data_cat.shape[1], activation = 'sigmoid')
decoded = decoding_layer(encoded)

autoencoder = Model(input_, decoded)
encoder = Model(input_, encoded)

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(data_train, data_train, epochs=50, validation_data=(data_test, data_test), verbose=False)

# print(data_test.iloc[1].values.reshape(data_test.shape[1],1).shape)
# print(data_test)
autoencoded_data = decoder.predict(encoder.predict(data_test)).round(0)
print(autoencoder.get_weights()[-2])
print(type(autoencoded_data))
print(type(data_test.values))
print(np.sum(np.sum((abs(autoencoded_data - data_test.values)), axis=1)))

(303, 13)
(303,)
[[-3.01661313e-01  6.62090003e-01 -4.11581755e-01  4.74634409e-01
  -3.38076726e-02  2.45603681e-01  2.32952029e-01 -9.65954661e-02
   2.20368698e-01 -2.61301696e-01  3.10562700e-01 -3.38573784e-01
   3.44081759e-01]
 [ 1.11989349e-01 -1.43439978e-01  3.11427400e-03 -2.85573512e-01
   2.85463240e-02 -1.55123249e-01  3.14072054e-03 -7.56887794e-02
  -1.24019630e-01 -6.63599193e-01  2.18883544e-01  5.65915763e-01
  -7.27179348e-01]
 [-1.22202910e-01 -6.93527758e-02  9.93300974e-02  1.26315832e-01
  -1.51919678e-01 -2.95511223e-02 -2.33785156e-02 -3.25395525e-01
   1.99739501e-01  8.46362948e-01 -8.44541013e-01 -1.02210507e-01
   4.14442718e-01]
 [ 2.89429694e-01  3.25679839e-01 -5.95579267e-01 -2.58272231e-01
  -4.79031533e-01 -4.18461055e-01  2.97389776e-01 -6.54131696e-02
  -1.76671147e-01  3.53872955e-01 -2.26733580e-01  1.62143663e-01
  -1.80221677e-01]
 [ 1.92813892e-02  1.01137406e-03 -3.89686853e-01  1.93303004e-02
   4.16743150e-03  6.51056826e-01 -3.45443130e-01

In [ ]:
tf.keras.backend.clear_session
tf.reset_default_graph()
data = pd.read_csv('heart-disease-uci.zip')
cat_col = ['cp', 'slope', 'thal', 'target']
data_cat = data[cat_col]
data_cat = oneHotEnc(data_cat, cat_col)
dummy_labels = np.ones([data_cat.shape[0]])
print(data_cat.shape)
print(dummy_labels.shape)
data_train, data_test, _, __, = train_test_split(data_cat, dummy_labels, test_size = 0.1)

encoding_dim = 40
input_ = Input(shape=(data_cat.shape[1], ))
encoded = Dense(encoding_dim, activation='relu')(input_)
decoding_layer = Dense(data_cat.shape[1], activation = 'sigmoid')
decoded = decoding_layer(encoded)

autoencoder = Model(input_, decoded)
encoder = Model(input_, encoded)

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(data_train, data_train, epochs=50, validation_data=(data_test, data_test), verbose=False)

# print(data_test.iloc[1].values.reshape(data_test.shape[1],1).shape)
# print(data_test)
# decoder.predict(encoder.predict(data_test)).round(0)
# print(decoder.predict(encoder.predict(data_test.iloc[1].values.reshape(data_test.shape[1]))))

# print(autoencoder.layers[-1].get_weights()[0])
##############################################################
def my_innit(shape, dtype=None):
    a = autoencoder.get_weights()[-2]
    return a

def my_binnit(shape, dtype=None):
    b = autoencoder.get_weights()[-1]
    return b

num_points = encoding_dim
print(num_points)
gen_input_size = 30
optimizer = Adam(0.0002, 0.9)
batch_size = int(data_cat.shape[0]*0.20)
epochs = 5001
steps_per_epoch = 5
def generator():
    generator = Sequential()
    
    generator.add(Dense(100, input_dim=gen_input_size))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(60, input_dim=gen_input_size))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(encoding_dim))
#     generator.add(BatchNormalization())
#     generator.add(LeakyReLU(0.2))
    
#     generator.add(Dense(encoding_dim))
    generator.add(Dense(data_cat.shape[1], kernel_initializer = my_innit, bias_initializer = my_binnit, trainable=False, activation='sigmoid'))
    
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator

def discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(100, input_dim=data_cat.shape[1]))
    discriminator.add(BatchNormalization())
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(50))
    discriminator.add(BatchNormalization())
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss = 'binary_crossentropy', optimizer=optimizer)
    
    return discriminator

discriminator = discriminator()
generator = generator()

discriminator.trainable = False

gan_input = Input(shape=(gen_input_size,))
generated_patient = generator(gan_input)
gan_output = discriminator(generated_patient)

gan = Model(gan_input, gan_output)

gan.compile(loss='binary_crossentropy', optimizer=optimizer)
start = time.time()
for epoch in range(epochs):
    for steps in range(steps_per_epoch):
        noise = sample_z(batch_size, gen_input_size)
        fake_x = generator.predict(noise)
        real_x = data_cat.sample(n=batch_size)
        x = np.concatenate((real_x, fake_x))
        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9
        d_loss = discriminator.train_on_batch(x, disc_y)
        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)
        
    if epoch%1000 == 0:
        print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(epoch,d_loss,g_loss))
print((time.time()-start)/60)

(303, 13)
(303,)
40


/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Iterations: 0	 Discriminator loss: 0.8587	 Generator loss: 0.6059
Iterations: 1000	 Discriminator loss: 0.1627	 Generator loss: 2.1480


In [65]:
a = generator.predict(sample_z(1, gen_input_size))
cohort_size = 100
for i in range(cohort_size):
    a = np.concatenate((a, generator.predict(sample_z(1, gen_input_size))), axis=0)
# print(data_cat.shape[1])
data = a
data_df = pd.DataFrame(data=data, columns = data_cat.columns.values)
# print(data_df)
# print(data_test)
for i in data_df.columns.values:
    print(i, ((np.sum(data_df[i].values.round(0)))))
print("#######################################################################")
for i in data_df.columns.values:
    print(i, ((np.sum(data_test[i].values)/data_test.shape[0])*100).round(2))

['cp']_0 2.0
['cp']_1 47.0
['cp']_2 54.0
['cp']_3 0.0
['slope']_0 1.0
['slope']_1 59.0
['slope']_2 44.0
['thal']_0 0.0
['thal']_1 0.0
['thal']_2 0.0
['thal']_3 101.0
['target']_0 99.0
['target']_1 0.0
#######################################################################
['cp']_0 41.94
['cp']_1 32.26
['cp']_2 25.81
['cp']_3 0.0
['slope']_0 9.68
['slope']_1 38.71
['slope']_2 51.61
['thal']_0 0.0
['thal']_1 6.45
['thal']_2 41.94
['thal']_3 51.61
['target']_0 51.61
['target']_1 48.39


In [42]:
data = pd.read_csv('heart-disease-uci.zip')
data_process = False
if data_process:    
    cont_ls = ['age', 'trestbps', 'chol', 'thalach','oldpeak']
    data = minMaxNormalise(data, cont_ls)
    cat_col = ['cp', 'slope', 'thal']
    data = oneHotEnc(data, cat_col)

target_df = data['target']
variables = data.drop(['target'], axis=1)

data_train, data_test, label_train, label_test = train_test_split(variables, target_df, test_size=0.20)
gan_data = pd.concat([data_train, label_train], axis=1, sort=False)

os.environ['KMP_DUPLICATE_LIB_OK']='True'
tf.reset_default_graph()
tf.keras.backend.clear_session
start = time.time()
num_points = gan_data.shape[1]
print(num_points)
gen_input_size = 100
optimizer = Adam(0.0002, 0.9)
batch_size = data_train.shape[0]
epochs = 50001
steps_per_epoch = 1
def generator():
    generator = Sequential()
    
    generator.add(Dense(500, input_dim=gen_input_size))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(200))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(50))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(num_points))
    
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator

def discriminator():
    discriminator = Sequential()
    
    discriminator.add(Dense(200, input_dim=num_points))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(50, input_dim=num_points))
    discriminator.add(LeakyReLU(0.2))
    
#     discriminator.add(Dense(50, input_dim=num_points))
#     discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss = 'binary_crossentropy', optimizer=optimizer)
    
    return discriminator

discriminator = discriminator()
generator = generator()

discriminator.trainable = False

gan_input = Input(shape=(gen_input_size,))
generated_patient = generator(gan_input)
gan_output = discriminator(generated_patient)

gan = Model(gan_input, gan_output)

gan.compile(loss='binary_crossentropy', optimizer=optimizer)

for epoch in range(epochs):
    for steps in range(steps_per_epoch):
        noise = sample_z(batch_size, gen_input_size)
        fake_x = generator.predict(noise)
        real_x = gan_data.sample(n=batch_size)
        x = np.concatenate((real_x, fake_x))
        disc_y = np.zeros(2*batch_size)
        disc_y[:batch_size] = 0.9
        d_loss = discriminator.train_on_batch(x, disc_y)
        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)
        
    if epoch%1000 == 0:
        print((time.time()-start)/60)
        print("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(epoch,d_loss,g_loss))

14


/opt/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0.07364900509516398
Iterations: 0	 Discriminator loss: 1.0702	 Generator loss: 0.7048
0.710802686214447
Iterations: 1000	 Discriminator loss: 0.3579	 Generator loss: 2.9466
1.3496827999750773
Iterations: 2000	 Discriminator loss: 0.3403	 Generator loss: 2.2525
1.9891611377398173
Iterations: 3000	 Discriminator loss: 0.2877	 Generator loss: 2.5429
2.6289148847262065
Iterations: 4000	 Discriminator loss: 0.3638	 Generator loss: 2.6033
3.268787682056427
Iterations: 5000	 Discriminator loss: 1.0531	 Generator loss: 2.4568
3.9091056187947593
Iterations: 6000	 Discriminator loss: 0.8747	 Generator loss: 0.9968
4.563680469989777
Iterations: 7000	 Discriminator loss: 0.3759	 Generator loss: 2.0373
5.207132951418559
Iterations: 8000	 Discriminator loss: 0.2806	 Generator loss: 2.8732
5.849877270062764
Iterations: 9000	 Discriminator loss: 0.3487	 Generator loss: 1.9284
6.50902335246404
Iterations: 10000	 Discriminator loss: 0.4468	 Generator loss: 1.8486
7.1528817017873125
Iterations: 11000	 Di

In [43]:
a = generator.predict(sample_z(1, gen_input_size))
cohort_size = 1000
for i in range(cohort_size):
    a = np.concatenate((a, generator.predict(sample_z(1, gen_input_size))), axis=0)
print(a.shape)
print(gan_data['target'].describe())
b = pd.DataFrame(a, columns=gan_data.columns.values.tolist())
target = b['target']
variables = b.drop(['target'], axis=1)
# print(target.head(10))
target_ls = []
target = target.round(0)
for i in target:
    if i>=1:
        target_ls.append(1)
    else:
        target_ls.append(0)
# print(target.describe())
# print(gan_data.sample(n=1))
total = 0
for i in target_ls:
    if i == 1:
        total = total + 1
        
total_l = 0
for i in label_test:
    if i == 1:
        total_l = total_l + 1
print(total)

print("Event rate in generated cohort: %.2f" %(total/cohort_size))
print("Event rate in test cohort: %.2f" %(total_l/len(label_test)))
print(len(label_test))
print(total_l)
print(b.head(10))
logreg = LogisticRegression()
logreg.fit(variables, target_ls)
accuracy = round(logreg.score(data_test, label_test)*100, 2)
print(accuracy)
print(confusion_matrix(label_test, logreg.predict(data_test)))
print(label_test.describe())

(1001, 14)
count    242.000000
mean       0.528926
std        0.500197
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: target, dtype: float64
0
Event rate in generated cohort: 0.00
Event rate in test cohort: 0.61
61
37
          age       sex        cp    trestbps        chol       fbs   restecg  \
0   99.684799  2.990803  1.109957  217.372803  339.374847  1.112384  0.166245   
1  106.844696  3.159410  1.204707  232.597748  363.650482  1.212611  0.181576   
2  116.297806  3.421916  1.315457  252.955475  395.680878  1.336409  0.201762   
3  101.011963  3.015105  1.133956  220.234299  343.896088  1.127637  0.173851   
4  102.971252  3.062208  1.160759  224.207687  350.319977  1.170142  0.177121   
5  110.840881  3.243651  1.269091  240.856262  376.932373  1.275178  0.184759   
6  109.073158  3.222993  1.226477  237.557419  371.303375  1.233913  0.186988   
7  108.227135  3.157053  1.248366  235.001465  367.670837  1.255879  0.1752

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0